# Occupancy detection

### A multivariate time series example

In this tutorial, you will learn how to apply getML to multivariate time series. It also demonstrates how to use getML's [high-level interface for hyperparameter tuning](https://docs.getml.com/latest/user_guide/hyperopt/hyperopt.html#tuning-routines).

Summary:

- Prediction type: __Binary classification__
- Domain: __Energy__
- Prediction target: __Room occupancy__
- Source data: __1 table, 32k rows__
- Population size: __32k__

_Author: Dr. Johannes King_

## Background

Many data scientists and AutoML tools use propositionalization methods for feature engineering. These propositionalization methods usually work as follows:

- Generate a large number of more-or-less hard-coded features
- Use feature selection to pick a percentage of these features

getML's FastProp is an implementation of a propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast our propositionalization approach FastProp is. To this end, we will benchmark getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

Our use case is a public domain data set for predicting room occupancy from sensor data. The results achieved using getML outperform all published results on this data set. Note that this is not only a neat use case for machine learning algorithms, but a real-world application with tangible consequences: If room occupancy is known with sufficient certainty, it can be applied to the control systems of a building. Such as system can reduce the energy consumption by [up to 50 %](https://ieeexplore.ieee.org/document/7566062). Fot further details about the data set refer to [link to the occupancy notebook](#).

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set-up your session:

In [13]:
import datetime
import os
from urllib import request
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("seaborn")
%matplotlib inline

import getml

from utils import FTTimeSeriesBuilder, TSFreshBuilder

print(f"getML API version: {getml.__version__}\n")

getml.engine.set_project("occupancy")

getML API version: 0.16.0


Connected to project 'occupancy'


## 1. Loading data


The data set can be downloaded directly from GitHub. It is conveniently separated into a train, a validation and a testing set. This allows us to directly benchmark our results against the results of the original paper later.

In [14]:
data_test, data_train, data_validate = getml.datasets.load_occupancy(
    roles=True
).values()

The train set looks like this:

In [15]:
data_train

Name,date,Occupancy,Temperature,Humidity,Light,CO2,HumidityRatio
Role,time_stamp,target,numerical,numerical,numerical,numerical,numerical
Units,"time stamp, comparison only",,,,,,
0,2015-02-04 17:51:00,1,23.18,27.272,426,721.25,0.004793
1,2015-02-04 17:51:59,1,23.15,27.2675,429.5,714,0.004783
2,2015-02-04 17:53:00,1,23.15,27.245,426,713.5,0.004779
3,2015-02-04 17:54:00,1,23.15,27.2,426,708.25,0.004772
4,2015-02-04 17:55:00,1,23.1,27.2,426,704.5,0.004757
,...,...,...,...,...,...,...
8138,2015-02-10 09:29:00,1,21.05,36.0975,433,787.25,0.005579
8139,2015-02-10 09:29:59,1,21.05,35.995,433,789.5,0.005563


## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

We use all possible aggregations. Because tsfresh and featuretools are single-threaded, we limit our FastProp algorithm to one thread as well, to ensure a fair comparison.

In [16]:
population = getml.data.Placeholder("population")
peripheral = getml.data.Placeholder("peripheral")

population.join(
    peripheral,
    time_stamp="date",
    # We want our time series features to only use
    # data from the last 15 minutes
    memory=getml.data.time.minutes(15),
    # Our forecast horizon is 0.
    # We do not predict the future, instead we infer
    # the present state from current and past sensor data.
    horizon=0.0,
    # We do not allow the time series features
    # to use target values from the past.
    allow_lagged_targets=False,
)

population

feature_learner = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    aggregation=getml.feature_learning.FastPropModel.agg_sets.All,
    num_threads=1
)

Next, we create the pipeline. In contrast to our usual approach, we create _two pipelines_ in
this notebook. One for feature learning (suffix `_fl`) and one for predicition (suffix `_pr`).
This allows for a fair comparison of runtimes.

In [17]:
pipe_fp_fl = getml.pipeline.Pipeline(
    feature_learners=[feature_learner],
    peripheral=[peripheral],
    population=population,
    tags=["feature learning", "fastprop"],
)

In [18]:
pipe_fp_fl.check(data_train, [data_train])

Checking data model...
OK.


The wrappers around featuretools and tsfresh fit on the training set and then return the training features. We therefore measure the time it takes getML's FastProp algorithm to fit on the training set and create the training features.

In [19]:
begin = time.time()

pipe_fp_fl.fit(data_train, [data_train])

fastprop_train = pipe_fp_fl.transform(
    data_train, 
    [data_train], 
    df_name="fastprop_train"
)

end = time.time()

fastprop_runtime = datetime.timedelta(seconds=end - begin)

Checking data model...
OK.

FastProp: Trying 194 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:0.139895


FastProp: Building features...
[========================================] 100%



In [20]:
fastprop_test = pipe_fp_fl.transform(data_test, [data_test], df_name="fastprop_test")


FastProp: Building features...
[========================================] 100%



Now we create a dedicated prediction pipeline and provide the fast prop features
(contrained in `fastprop_train` and `fastprop_test`.)

In [21]:
predictor = getml.predictors.XGBoostClassifier()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [22]:
pipe_fp_pr.check(fastprop_train)

pipe_fp_pr.fit(fastprop_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_163' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_165' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_168' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_174' in POPULATION__STAGING_T

Checking data model...
Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.142093



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/Z4KeQf/0/

In [23]:
pipe_fp_pr.score(fastprop_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-17 16:45:53,fastprop_train,Occupancy,0.9998,1.,0.004514
1,2021-05-17 16:45:53,fastprop_test,Occupancy,0.9936,0.9986,0.025329


### 2.2 Propositionalization with featuretools

In [24]:
dfs_pandas = {}

for df in getml.project.data_frames:
    dfs_pandas[df.name] = df.to_pandas()
    dfs_pandas[df.name]["id"] = 1

In [25]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(minutes=0),
    memory=pd.Timedelta(minutes=15),
    column_id="id",
    time_stamp="date",
    target="Occupancy",
)

The `FTTimeSeriesBuilder` provides a `fit` method that is designed to be equivilant to
to the `fit` method of the predictorless getML pipeline above.

In [26]:
featuretools_train = ft_builder.fit(dfs_pandas["train"])
featuretools_test = ft_builder.transform(dfs_pandas["test"])

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=data_train.roles
)
df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=data_train.roles
)

df_featuretools_train.set_role(
    df_featuretools_train.unused_names, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.unused_names, getml.data.roles.numerical
)

featuretools: Trying features...


/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Time taken: 0h:3m:15.06355



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [27]:
predictor = getml.predictors.XGBoostClassifier()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

In [28]:
pipe_ft_pr.check(df_featuretools_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...


In [29]:
pipe_ft_pr.fit(df_featuretools_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:2.239418



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/c9uWrA/0/

In [30]:
pipe_ft_pr.score(df_featuretools_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-17 16:53:08,featuretools_train,Occupancy,0.999,1.,0.006113
1,2021-05-17 16:53:09,featuretools_test,Occupancy,0.9937,0.998,0.026789


### 2.3 Propositionalization with tsfresh

In [31]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, memory=15, column_id="id", time_stamp="date", target="Occupancy"
)

tsfresh_train = tsfresh_builder.fit(dfs_pandas["train"])
tsfresh_test = tsfresh_builder.transform(dfs_pandas["test"])

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=data_train.roles
)
df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=data_train.roles
)

df_tsfresh_train.set_role(df_tsfresh_train.unused_names, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.unused_names, getml.data.roles.numerical)

/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:456: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "
Feature Extraction: 100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


Time taken: 0h:0m:59.594586



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:456: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "
Feature Extraction: 100%|██████████| 20/20 [00:22<00:00,  1.10s/it]


In [32]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

In [33]:
pipe_tsf_pr.check(df_tsfresh_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...


In [34]:
pipe_tsf_pr.fit(df_tsfresh_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:1.622233



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/O5va8K/0/

In [35]:
pipe_tsf_pr.score(df_tsfresh_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-17 16:55:05,tsfresh_train,Occupancy,0.9993,1.,0.005947
1,2021-05-17 16:55:05,tsfresh_test,Occupancy,0.9932,0.9982,0.032016


### 3. Comparison

In [36]:
num_features = dict(
    fastprop=194,
    featuretools=122,
    tsfresh=55,
)

runtime_per_feature = [
    fastprop_runtime / num_features["fastprop"],
    ft_builder.runtime / num_features["featuretools"],
    tsfresh_builder.runtime / num_features["tsfresh"],
]

features_per_second = [1.0/r.total_seconds() for r in runtime_per_feature]

speedup_per_feature = [r/runtime_per_feature[0] for r in runtime_per_feature]

comparison = pd.DataFrame(
    dict(
        runtime=[fastprop_runtime, ft_builder.runtime, tsfresh_builder.runtime],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        speedup=[1, ft_builder.runtime/fastprop_runtime, tsfresh_builder.runtime/fastprop_runtime],
        speedup_per_feature=speedup_per_feature,
        auc=[pipe_fp_pr.auc, pipe_ft_pr.auc, pipe_tsf_pr.auc],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [38]:
comparison

,runtime,num_features,features_per_second,speedup,speedup_per_feature,auc
getML: FastProp,0 days 00:00:04.108123,194,47.223272,1.000000,1.000000,0.998594
featuretools,0 days 00:03:15.063550,122,0.625437,47.482403,75.504439,0.997984
tsfresh,0 days 00:00:59.594586,55,0.922903,14.506524,51.168209,0.998226


In [39]:
# export for further use
comparison.to_csv("comparisons/occupancy.csv")

### Why is FastProp so fast?

First, FastProp hugely benefits from getML's custom-built C++-native in-memory database engine. The engine is highly optimized for working with relational data structures and makes use of information about the relational structure of the data to efficiently store and carry out computations on such data. This matters in particular for time series where we [relate the current observation to a certain number of observations from the past](https://docs.getml.com/latest/user_guide/data_model/data_model.html#time-series): Other libraries have to deal explicitly with this inherent structure of (multivariate) time series; and such explicit transformations are costly, in terms of consumption of both, memory and computational resources. All operations on data stored in getML's engine benefit from implementations in modern C++. Further, we are taking advantage of functional design patterns where all column-based operations are evaluated lazily. So, for example, aggregations are carried out only on rows that matter (taking into account even complex conditions that might span multiple tables in the relational model). Duplicate operations are reduced to a bare minimum by keeping track of the relational data model. In addition to the mere advantage in performance, FastProp, by building on an abstract data model, also has an edge in memory consumption based on the abstract database design, the reliance on efficient storage patterns (utilizing pointers and indices) for concrete data, and by taking advantage of functional design patterns and lazy computations. This allows working with data sets of substantial size even without falling back to distributed computing models.

## 4. Conclusion

This tutorial demonstrates that relational learning is a powerful tool for time series. We able to outperform the benchmarks for a scientific paper on a simple public domain time series data set using relatively little effort.

If you want to learn more about getML, check out the [official documentation](https://getml.com/product).

# Next Steps

This tutorial went through the basics of applying getML to multirvariate time series and hyperparameter tuning.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.